
## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
file_location = "/FileStore/tables/tips-1.csv"
file_type = "csv"

# read data csv tips
df = spark.read.csv(file_location, header=True, inferSchema=True)

In [0]:
# print schema
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [0]:
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [0]:
df.groupBy('sex').count().show()

+------+-----+
|   sex|count|
+------+-----+
|Female|   87|
|  Male|  157|
+------+-----+



In [0]:
df.columns

Out[7]: ['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size']

In [0]:
## handling categorical feature
from pyspark.ml.feature import StringIndexer


In [0]:
indexer = StringIndexer(inputCols=['sex', 'smoker', 'day', 'time'],outputCols=['sex_index', 'smoker_index', 'day_index', 'time_index'])
df_r = indexer.fit(df).transform(df)
df_r.show()

+----------+----+------+------+---+------+----+---------+------------+---------+----------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_index|smoker_index|day_index|time_index|
+----------+----+------+------+---+------+----+---------+------------+---------+----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|      1.0|         0.0|      1.0|       0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|      0.0|         0.0|      1.0|       0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|      0.0|         0.0|      1.0|       0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|      0.0|         0.0|      1.0|       0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|      1.0|         0.0|      1.0|       0.0|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|      0.0|         0.0|      1.0|       0.0|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|      0.0|         0.0|      1.0|       0.0|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|      0.0|         0.0|      1.0|

In [0]:
df_r.columns

Out[10]: ['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_index',
 'smoker_index',
 'day_index',
 'time_index']

In [0]:
from pyspark.ml.feature import VectorAssembler

feature_assembler = VectorAssembler(
    inputCols=['tip','size','sex_index', 'smoker_index', 'day_index', 'time_index'],
    outputCol="Independent Feature"
    )

output = feature_assembler.transform(df_r)

In [0]:
output.show()

+----------+----+------+------+---+------+----+---------+------------+---------+----------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|sex_index|smoker_index|day_index|time_index| Independent Feature|
+----------+----+------+------+---+------+----+---------+------------+---------+----------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|      1.0|         0.0|      1.0|       0.0|[1.01,2.0,1.0,0.0...|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|      0.0|         0.0|      1.0|       0.0|[1.66,3.0,0.0,0.0...|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|      0.0|         0.0|      1.0|       0.0|[3.5,3.0,0.0,0.0,...|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|      0.0|         0.0|      1.0|       0.0|[3.31,2.0,0.0,0.0...|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|      1.0|         0.0|      1.0|       0.0|[3.61,4.0,1.0,0.0...|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|      0.0|         0.0|      1.0|       0.0|[4.71

In [0]:
output.columns

Out[15]: ['total_bill',
 'tip',
 'sex',
 'smoker',
 'day',
 'time',
 'size',
 'sex_index',
 'smoker_index',
 'day_index',
 'time_index',
 'Independent Feature']

In [0]:
finalized_data = output.select("Independent Feature",'total_bill')
finalized_data.show(10)

+--------------------+----------+
| Independent Feature|total_bill|
+--------------------+----------+
|[1.01,2.0,1.0,0.0...|     16.99|
|[1.66,3.0,0.0,0.0...|     10.34|
|[3.5,3.0,0.0,0.0,...|     21.01|
|[3.31,2.0,0.0,0.0...|     23.68|
|[3.61,4.0,1.0,0.0...|     24.59|
|[4.71,4.0,0.0,0.0...|     25.29|
|[2.0,2.0,0.0,0.0,...|      8.77|
|[3.12,4.0,0.0,0.0...|     26.88|
|[1.96,2.0,0.0,0.0...|     15.04|
|[3.23,2.0,0.0,0.0...|     14.78|
+--------------------+----------+
only showing top 10 rows



In [0]:
## train data
from pyspark.ml.regression import LinearRegression

train_data, test_data = finalized_data.randomSplit([0.75,0.25])
regressor = LinearRegression(featuresCol='Independent Feature', labelCol='total_bill')
regressor = regressor.fit(train_data)

In [0]:
regressor.coefficients

Out[22]: DenseVector([3.3891, 3.3793, -0.4035, 2.4381, -0.4506, -0.8616])

In [0]:
regressor.intercept

Out[23]: 0.8498917360024552

In [0]:
pred_results = regressor.evaluate(test_data)

In [0]:
pred_results.predictions.show()

+--------------------+----------+------------------+
| Independent Feature|total_bill|        prediction|
+--------------------+----------+------------------+
| (6,[0,1],[2.0,2.0])|     13.37|14.386764394775362|
|(6,[0,1],[2.72,2.0])|     13.28|16.826921612214488|
| (6,[0,1],[3.0,2.0])|      14.0|17.775871641218593|
| (6,[0,1],[3.0,4.0])|     20.45| 24.53452980710504|
|(6,[0,1],[3.27,2.0])|     17.78|18.690930597758264|
|(6,[0,1],[3.39,2.0])|     11.61|19.097623467331452|
| (6,[0,1],[4.3,2.0])|      21.7| 22.18171106159479|
|[1.17,2.0,0.0,1.0...|     32.83|14.011860750753591|
|[1.5,2.0,0.0,0.0,...|     19.08|10.929443786953136|
|[1.5,2.0,0.0,1.0,...|     12.03|  13.7784922941637|
|[1.96,2.0,0.0,0.0...|     15.04|13.800608822278914|
|[2.0,2.0,0.0,0.0,...|      8.77|13.936173112136643|
|[2.0,2.0,0.0,0.0,...|     13.13|13.936173112136643|
|[2.0,2.0,0.0,0.0,...|     13.81|13.936173112136643|
|[2.0,2.0,0.0,1.0,...|     13.81|16.824819765301473|
|[2.0,2.0,0.0,1.0,...|     22.67|16.8248197653

In [0]:
pred_results.meanAbsoluteError,pred_results.meanSquaredError

Out[26]: (4.325818637186084, 33.55257163639996)